<a href="https://colab.research.google.com/github/NguyenThuan215/int3404-3-simpleORC/blob/main/int3404.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import time
import sys
import cv2

import random
import torch
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
!git clone https://github.com/NguyenThuan215/int3404-3-simpleORC/

Cloning into 'int3404-3-simpleORC'...
remote: Enumerating objects: 687, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 687 (delta 4), reused 38 (delta 4), pack-reused 649
Receiving objects: 100% (687/687), 654.81 MiB | 18.77 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Checking out files: 100% (644/644), done.


In [3]:
def imshow(img):
    img = img / 2 + 0.5     
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


class HoadonOCR:

  def __init__(self):
    # Init parameters, load model here
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    self.modeldir = "/content/int3404-3-simpleORC/model.pth"
    self.model = self.loadModel(modeldir=self.modeldir, device=self.device)
    self.labels = ['highlands', 'others', 'phuclong', 'starbucks']

  # TODO: implement find label
  def find_label(self, img):
    image = self.transformImage(img)
    image = image.to(self.device)
    self.model.eval()
    with torch.no_grad():
      output = self.model(image.float())
      _, predicted = torch.max(output, dim=1)
      print(output, self.labels[int(predicted.cpu().numpy())])

    return self.labels[int(predicted.cpu().numpy())]

  def loadModel(self, modeldir, device):
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=512, out_features=4, bias=True)
    model.load_state_dict(torch.load(modeldir, map_location=device))
    model.to(device)
    return model
  
  def transformImage(self, image):
    img_size = 224
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    img_BGR = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # convert BGR to RGB
    img_resize = cv2.resize(img_BGR, (img_size,img_size)) # resize
    img_resize = np.array(img_resize, dtype="uint8") # to ndarray
    image = img_resize/255 # into 0-1
    image_tensor = torch.tensor(image) # convert to tensor
    for color in range(3): # nomalize
      image_tensor[:,:,color] = (image_tensor[:,:,color] - mean[color])/std[color]

    image_tensor = np.transpose(image_tensor, (2, 0, 1)) # transpose
    image_tensor = image_tensor.unsqueeze(0) # add dimension
    return image_tensor

    


In [4]:
if __name__ == "__main__":

  input_folder = '/content/int3404-3-simpleORC/sampledata'
  label_file = '/content/int3404-3-simpleORC/labels.csv'

  df_labels = pd.read_csv(label_file)
  img_name = df_labels['img_name'].values.tolist()
  img_labels = df_labels['label'].values.tolist()
  img_to_label = dict([(img_name[i], img_labels[i]) for i in range(len(img_name))])

  start_time = time.time()
  model = HoadonOCR()
  init_time = time.time() - start_time
  print("Run time in: %.2f s" % init_time)

  list_files = os.listdir(input_folder)
  print("Total test images: ", len(list_files))
  fail_process = 0
  cnt_predict = 0

  start_time = time.time()
  for filename in list_files:
    img = cv2.imread(os.path.join(input_folder, filename))
    print(filename, img.shape, end = " ")
    try:
      label = model.find_label(img)
    except:
      label = -1

    if img_to_label[filename] == label:
      cnt_predict += 1
    elif label == -1:
      print(filename)
      fail_process += 1

  run_time = time.time() - start_time

  print("Ket qua dung: %i/%i" % (cnt_predict, len(list_files)))
  print("Loi: %i" % fail_process)
  print("Score = %.2f" % (10.*cnt_predict/len(list_files)))
  print("Run time in: %.2f s" % run_time)
  cv2.waitKey(0)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip


Run time in: 1.62 s
Total test images:  10
img10.jpeg (1024, 1024, 3) tensor([[-11.2745,  18.5592,  -6.0317,  -1.1337]]) others
img2.jpeg (966, 800, 3) tensor([[ 6.2059, -5.0238, -2.1173,  0.6191]]) highlands
img7.jpeg (768, 1024, 3) tensor([[-9.1351,  8.7940, -0.1771,  0.5555]]) others
img6.jpeg (966, 800, 3) tensor([[-10.0128,  -2.0488,  -1.7373,  12.8300]]) starbucks
img4.jpeg (1067, 800, 3) tensor([[-8.5241, -1.8575,  8.5299,  1.6615]]) phuclong
img9.jpeg (931, 1242, 3) tensor([[-17.3158,  20.0756,  -1.6189,  -1.0640]]) others
img5.jpeg (966, 800, 3) tensor([[-6.0900, -2.9504, 10.3984, -1.3515]]) phuclong
img8.jpeg (701, 526, 3) tensor([[-0.8500,  0.0864,  0.1131,  0.5032]]) starbucks
img1.jpeg (1067, 800, 3) tensor([[ 8.6434, -3.0901, -1.7657, -4.0277]]) highlands
img3.jpeg (1067, 800, 3) tensor([[ 8.6434, -3.0901, -1.7657, -4.0277]]) highlands
Ket qua dung: 9/10
Loi: 0
Score = 9.00
Run time in: 1.27 s
